<a href="https://colab.research.google.com/github/shah-zeb-naveed/autogen-udemy-course/blob/main/autobuild_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyautogen[autobuild] --upgrade

In [31]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

import autogen
from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent
)
from autogen.agentchat.contrib import agent_builder
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor


config_file_or_env = "OAI_CONFIG_LIST"
config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
default_llm_config = {'temperature': 0}

In [16]:
from autogen.agentchat.contrib.agent_builder import AgentBuilder

builder = AgentBuilder(
    config_file_or_env="OAI_CONFIG_LIST",
    builder_model=config_list[0]['model'],
    agent_model=config_list[0]['model']
)

In [17]:
building_task = """Find a paper on arxiv by programming, and
analyze its application in some domain. For example, find a latest
paper about gpt-4 on arxiv and find its potential applications in software."""

building_task

'Find a paper on arxiv by programming, and \nanalyze its application in some domain. For example, find a latest \npaper about gpt-4 on arxiv and find its potential applications in software.'

In [19]:
agent_list, agent_configs = builder.build(building_task, default_llm_config, coding=True)

==> Generating agents...
['1._research_scientist\n2._software'] are generated.
==> Generating system message...
Preparing system message for 1._research_scientist
2._software
==> Generating description...
Preparing description for 1._research_scientist
2._software
==> Creating agents...
Creating agent 1._research_scientist
2._software with backbone gpt-3.5-turbo...
Adding user console proxy...


In [22]:
print(agent_list)

import pprint

pprint.pp(agent_configs)

[<autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x7dba11f92bc0>, <autogen.agentchat.assistant_agent.AssistantAgent object at 0x7dba004f6da0>]
{'building_task': 'Find a paper on arxiv by programming, and \n'
                  'analyze its application in some domain. For example, find a '
                  'latest \n'
                  'paper about gpt-4 on arxiv and find its potential '
                  'applications in software.',
 'agent_configs': [{'name': '1._research_scientist\n2._software',
                    'model': 'gpt-3.5-turbo',
                    'system_message': 'You are now in a group chat. You need',
                    'description': "[POSITION's name] is a research scientist "
                                   'with'}],
 'coding': True,
 'default_llm_config': {'temperature': 0},
 'code_execution_config': {'last_n_messages': 2,
                           'work_dir': 'groupchat',
                           'use_docker': False,
                         

In [27]:
build_path = builder.save()

Building config saved to ./save_config_8338de6e0faeb1146dbda0d8019df755.json


In [28]:
builder_clone = AgentBuilder(config_file_or_env=config_file_or_env)
agent_list, agent_config = builder_clone.load(build_path)
pprint.pp(agent_config)

Loading config from ./save_config_8338de6e0faeb1146dbda0d8019df755.json
==> Creating agents...
Creating agent 1._research_scientist
2._software with backbone gpt-3.5-turbo...
Adding user console proxy...
{'building_task': 'Find a paper on arxiv by programming, and \n'
                  'analyze its application in some domain. For example, find a '
                  'latest \n'
                  'paper about gpt-4 on arxiv and find its potential '
                  'applications in software.',
 'agent_configs': [{'name': '1._research_scientist\n2._software',
                    'model': 'gpt-3.5-turbo',
                    'system_message': 'You are now in a group chat. You need',
                    'description': "[POSITION's name] is a research scientist "
                                   'with'}],
 'coding': True,
 'default_llm_config': {'temperature': 0},
 'code_execution_config': {'last_n_messages': 2,
                           'work_dir': 'groupchat',
                         

In [32]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False
)

autobuild_assistant = AssistantAgent(
    name="Autobuild Assistant",
    llm_config={"config_list": config_list},
)

def autobuild_reply(recipient, messages, sender, config):
    building_task = messages[-1]["content"]
    builder = AgentBuilder(
        config_file_or_env="OAI_CONFIG_LIST",
        builder_model=config_list[0]['model'],
        agent_model=config_list[0]['model']
    )

    agent_list, agent_configs = builder.build(building_task, default_llm_config, coding=True)

    # create group chat
    nested_group_chat = GroupChat(
        agents=agent_list,
        messages=[],
    )

    # create group chat manager
    manager = GroupChatManager(groupchat=nested_group_chat, llm_config={"config_list": config_list, "cache_seed": None})

    # solve task
    chat_res = agent_list[0].initiate_chat(
        manager, message=building_task, summary_method="reflection_with_llm"
    )

    # clear state of agents if current chat not needed for next user interaction
    #builder.clear_all_agents()

    return True, chat_res.summary


autobuild_assistant.register_reply([Agent, None], autobuild_reply)

with Cache.disk(cache_seed=41) as cache:
    user_proxy.initiate_chat(autobuild_assistant, message=building_task, max_turns=1)

user_proxy (to Autobuild Assistant):

Find a paper on arxiv by programming, and 
analyze its application in some domain. For example, find a latest 
paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
==> Generating agents...
['1._research_scientist\n2._software'] are generated.
==> Generating system message...
Preparing system message for 1._research_scientist
2._software
==> Generating description...
Preparing description for 1._research_scientist
2._software
==> Creating agents...
Creating agent 1._research_scientist
2._software with backbone gpt-3.5-turbo...
Adding user console proxy...
User_console_and_code_interpreter (to chat_manager):

Find a paper on arxiv by programming, and 
analyze its application in some domain. For example, find a latest 
paper about gpt-4 on arxiv and find its potential applications in software.

------------------------------------------------------

1._research_scientist


1._research_scientist
2._software (to chat_manager):

Hey there! That sounds like an interesting task.

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "'1._research_scientist\\n2._software' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.2.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}